- NN comprises of **layers(modules)** that perform operations on data
- `torch.nn`: provides all the components you need to build NN

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

**Get Device for Training**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


**Define the Class**
- define NN by subclassing `nn.Module`, and initialize NN layers in `__init__`
- every `nn.Module` subclass implents the operations on input data in the `forward` method

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

- create an instance of `NeuralNetwork`, move it to the `device`, and print its structure

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- pass the input data to use the model
- executes the model's `forward', along with some background operations
- do not call `model.forward()` directly!

- calling the model on the input returns a 2D tensor with dim=0 corresponding to each output of 10 **raw predicted values** for each **class**, and dim=1 corresponding to the individual values of each output
- we get the **prediction probabilities** by passing the **raw predicted values** through an instance of `nn.Softmax` module

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


**Model Layers**
- FashionMNIST model
  - take a sample minibatch of 3 images of size 28x28

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**
- initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values
- (minibatch dimension is maintained)

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear**
- linear layer is a module that applies a linear transformation on the input using its stored weights and biases (Wx + b)

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**
- non-linear activations are what create the complex mappings between the model's inputs and outputs
- they are applied after linear transformations to introduce nonlinearity, helping NN learn a wide variety of phenomena

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4709, -0.2810, -0.0382,  0.0285,  0.2996,  0.7960,  0.8344, -0.0950,
         -0.1476, -0.3799,  0.5232, -0.2065, -0.2733,  0.1762,  0.0556,  0.2360,
          0.1595, -0.2812,  0.2850, -0.0589],
        [-0.0461,  0.0861, -0.3667, -0.1153,  0.7376,  0.6836,  0.8744,  0.0334,
         -0.3473, -0.4491,  0.7331, -0.1406,  0.3271,  0.3025,  0.0689,  0.2456,
         -0.0961,  0.1034, -0.0709, -0.2043],
        [-0.2598,  0.1525,  0.1525, -0.2377,  0.4515,  0.5143,  0.4914,  0.3224,
          0.0148, -0.1914,  0.3541,  0.3023, -0.1461,  0.1104,  0.3134,  0.1959,
          0.0607, -0.1917,  0.2439, -0.1998]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0285, 0.2996, 0.7960, 0.8344, 0.0000, 0.0000,
         0.0000, 0.5232, 0.0000, 0.0000, 0.1762, 0.0556, 0.2360, 0.1595, 0.0000,
         0.2850, 0.0000],
        [0.0000, 0.0861, 0.0000, 0.0000, 0.7376, 0.6836, 0.8744, 0.0334, 0.0000,
         0.0000, 0.7331, 0.0000, 0.3271, 0.3025, 0.06

**nn.Sequential**
- an ordered container of modules
- the data is passed through all the modules in the same order as defined
- we can use sequential containers to put together a quick network like `seq_modules`

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10),
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

**nn.Softmax**
- the last linear layer of NN returns logits(raw values in [-INF, INF])
- logits are passed to `nn.Softmax` module
- logits are scaled to values [0, 1] representing the model's predicted probabilities for each class
- `dim` parameter indicates the dimension along which the values must sum to 1

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

**Model Parameters**
- many layers inside NN are parameterized (매개변수화)
- i.e., weights, biases that are optimized during training
- subclassing `nn.Module` automatically tracks all fields defined inside the model, and makes all parameters accessible using the model's `parameters()` or `named_parameters()` methods

In [ ]:
# iterate over each parameter, and print its size and a preview of its values
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0165,  0.0106, -0.0211,  ..., -0.0112,  0.0094,  0.0024],
        [-0.0269, -0.0344, -0.0025,  ..., -0.0006, -0.0063,  0.0097]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0255,  0.0211], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0305, -0.0117, -0.0205,  ...,  0.0115, -0.0010, -0.0183],
        [-0.0293,  0.0373, -0.0172,  ..., -0.0372,  0.0436,  0.0084]],
       device='cuda:0', grad_fn=<Sl